# Marketing Budget Allocation (Linear Programming)

In this project, I tackle a linear programming optimization problem that builds a simple marketing budget allocation strategy, namely how
much to invest in each advertisement platform (print, TV, Facebook, etc).

In [ ]:
import numpy as np
import gurobipy as gp
import pandas as pd
import csv

##Part 1 - Introduction and Scenario Setup


In [ ]:
# Import and Look at Data

file_path_index_data = "index_data.csv"
index_data = pd.read_csv(file_path_index_data)
file_path_ROI_data = "ROI_data.csv"
roi_data = pd.read_csv(file_path_ROI_data)
file_path_roi_mat = "roi_mat.csv"
roi_mat = pd.read_csv(file_path_roi_mat)


In [ ]:
index_data

,Unnamed: 0,Print,TV,SEO,AdWords,Facebook,LinkedIn,Instagram,Snapchat,Twitter,Email
0,January,1,13,25,37,49,61,73,85,97,109
1,February,2,14,26,38,50,62,74,86,98,110
2,March,3,15,27,39,51,63,75,87,99,111
3,April,4,16,28,40,52,64,76,88,100,112
4,May,5,17,29,41,53,65,77,89,101,113
5,June,6,18,30,42,54,66,78,90,102,114
6,July,7,19,31,43,55,67,79,91,103,115
7,August,8,20,32,44,56,68,80,92,104,116
8,September,9,21,33,45,57,69,81,93,105,117
9,October,10,22,34,46,58,70,82,94,106,118


In [ ]:
roi_data

,Platform,Print,TV,SEO,AdWords,Facebook,LinkedIn,Instagram,Snapchat,Twitter,Email
0,ROI,0.03,0.05,0.02,0.04,0.02,0.02,0.05,0.03,0.03,0.04
1,Second Firms ROI Estimate,0.05,0.02,0.02,0.04,0.04,0.05,0.03,0.02,0.04,0.03


In [ ]:
roi_mat

,Unnamed: 0,Print,TV,SEO,AdWords,Facebook,LinkedIn,Instagram,Snapchat,Twitter,Email
0,January,4.00,3.60,2.40,3.90,3.00,3.50,3.60,2.25,3.50,3.50
1,February,4.00,3.90,2.70,3.80,4.30,3.20,2.70,1.80,3.70,3.50
2,March,3.50,2.90,3.10,3.80,2.40,4.10,3.70,2.60,4.20,2.50
3,April,3.80,3.10,2.40,4.40,2.40,3.80,3.70,2.50,3.60,2.90
4,May,3.50,3.20,1.90,3.40,2.70,2.70,3.90,2.20,4.50,3.90
5,June,4.00,3.20,2.70,3.40,3.40,3.00,4.50,2.10,3.80,4.10
6,July,3.90,3.60,2.00,4.40,3.90,3.70,4.30,1.80,4.00,3.80
7,August,4.20,3.30,2.80,4.20,2.00,3.70,3.60,1.50,4.40,4.30
8,September,4.10,2.80,2.50,4.20,2.90,3.70,2.80,2.50,4.00,3.40
9,October,3.00,3.00,3.10,4.60,3.10,3.30,3.20,2.30,2.50,3.20


##Part 2 - Linear Program Formulation


In [ ]:
# Constraints

# Print + TV <= Facebook + Email
# Facebook + LinkedIn + Instagram + Snapchat + Twitter >= 2 * (SEO + AdWords)

individual_max = 3000000
Budget = 10000000
obj = roi_data.iloc[0, 1:]

In [ ]:
obj

Print       0.03
TV          0.05
SEO         0.02
AdWords     0.04
Facebook    0.02
LinkedIn    0.02
Instagram   0.05
Snapchat    0.03
Twitter     0.03
Email       0.04
Name: 0, dtype: object

In [ ]:
# Identifying the indices of each of the platforms in roi_data
# We are assuming that the platforms included in the constraints are always included in the data.
# We also assume that all the column names are spelled & capitalized correctly.

platforms = {}
columns = roi_data.iloc[:, 1:]
for column in columns:
    platforms[column] = columns.columns.get_loc(column)
print(platforms)

{'Print': 0, 'TV': 1, 'SEO': 2, 'AdWords': 3, 'Facebook': 4, 'LinkedIn': 5, 'Instagram': 6, 'Snapchat': 7, 'Twitter': 8, 'Email': 9}


##Part 3 - Optimal Budget Allocation


In [ ]:
# Objective: Maximize ROI
# 10 Decision variables, one for each marketing medium

Mod = gp.Model()

# Each investment is <= $3 Million
ModX = Mod.addMVar(len(obj), ub = individual_max)

# Print + TV <= Facebook + Email
ModCon_a = Mod.addConstr((ModX[platforms["Print"]] + ModX[platforms["TV"]]) <= (ModX[platforms["Facebook"]] + ModX[platforms["Email"]]))

# Facebook + LinkedIn + Instagram + Snapchat + Twitter >= 2 * (SEO + AdWords)
ModCon_b = Mod.addConstr((ModX[platforms["Facebook"]] + ModX[platforms["LinkedIn"]] + ModX[platforms["Instagram"]] + ModX[platforms["Snapchat"]] + ModX[platforms["Twitter"]]) >= (2 * (ModX[platforms["SEO"]] + ModX[platforms["AdWords"]])))

# Budget constraint of $10 Million
ModCon_budget = Mod.addConstr(gp.quicksum(ModX[i] for i in range(len(obj))) <= Budget)

# Maximize ROI
Mod.setObjective(gp.quicksum((obj[i]) * ModX[i] for i in range(len(obj))), sense = gp.GRB.MAXIMIZE)
Mod.Params.OutputFlag=0
Mod.optimize()

In [ ]:
Mod.x

[0.0, 3000000.0, 0.0, 1000000.0, 0.0, 0.0, 3000000.0, 0.0, 0.0, 3000000.0]

In [ ]:
print("\n" + "="*90 + "\n")

for platform in platforms:
    # Format and print each row of the objective analysis table
    print(f"{platform:<30} {Mod.x[platforms[platform]]:<25}")



Print                          0.0                      
TV                             3000000.0                
SEO                            0.0                      
AdWords                        1000000.0                
Facebook                       0.0                      
LinkedIn                       0.0                      
Instagram                      3000000.0                
Snapchat                       0.0                      
Twitter                        0.0                      
Email                          3000000.0                


In [ ]:
Mod.objVal

456000.0

##Part 4 - Comparison of ROI Data and Allocations


In [ ]:
obj2 = roi_data.iloc[1, 1:]

In [ ]:
# Same process as above but using second ROI data

Mod2 = gp.Model()

ModX2 = Mod2.addMVar(len(obj), ub = individual_max)

ModCon_a2 = Mod2.addConstr((ModX2[platforms["Print"]] + ModX2[platforms["TV"]]) <= (ModX2[platforms["Facebook"]] + ModX2[platforms["Email"]]))
ModCon_b2 = Mod2.addConstr((ModX2[platforms["Facebook"]] + ModX2[platforms["LinkedIn"]] + ModX2[platforms["Instagram"]] + ModX2[platforms["Snapchat"]] + ModX2[platforms["Twitter"]]) >= (2 * (ModX2[platforms["SEO"]] + ModX2[platforms["AdWords"]])))
ModCon_budget2 = Mod2.addConstr(gp.quicksum(ModX2[i] for i in range(len(obj2))) <= Budget)
Mod2.setObjective(gp.quicksum((obj2[i]) * ModX2[i] for i in range(len(obj2))), sense=gp.GRB.MAXIMIZE)
Mod2.Params.OutputFlag=0
Mod2.optimize()

In [ ]:
Mod2.x

[3000000.0, 0.0, 0.0, 1000000.0, 3000000.0, 3000000.0, 0.0, 0.0, 0.0, 0.0]

In [ ]:
print("\n" + "="*90 + "\n")

for platform in platforms:
    # Format and print each row of the objective analysis table
    print(f"{platform:<30} {Mod2.x[platforms[platform]]:<25}")



Print                          3000000.0                
TV                             0.0                      
SEO                            0.0                      
AdWords                        1000000.0                
Facebook                       3000000.0                
LinkedIn                       3000000.0                
Instagram                      0.0                      
Snapchat                       0.0                      
Twitter                        0.0                      
Email                          0.0                      


In [ ]:
Mod2.objVal

456000.0

##Part 5 - Sensitivity Analysis on ROI Data


In [ ]:
obj = roi_data.iloc[0, 1:]
obj

Print       0.03
TV          0.05
SEO         0.02
AdWords     0.04
Facebook    0.02
LinkedIn    0.02
Instagram   0.05
Snapchat    0.03
Twitter     0.03
Email       0.04
Name: 0, dtype: object

In [ ]:
obj2 = roi_data.iloc[1, 1:]
obj2

Print       0.05
TV          0.02
SEO         0.02
AdWords     0.04
Facebook    0.04
LinkedIn    0.05
Instagram   0.03
Snapchat    0.02
Twitter     0.04
Email       0.03
Name: 1, dtype: object

In [ ]:
# Assuming the first ROI data is correct, if you were to use the
# second allocation how much lower would the objective be relative to the optimal objective?

# From the results below, we can determine that if the second allocations were used on the first ROI data the
# objective value would be $204,000 lower

secondAllocation_firstROI = sum(obj[i] * Mod2.x[i] for i in range(len(obj2)))
diff = Mod.objVal - secondAllocation_firstROI
diff

204000.0

In [ ]:
# Assuming the second ROI data is correct, if you used the first
# allocation how much lower would the objective be relative to the optimal objective?

# From the results below, we can determine that if the first allocations were used on the second ROI data the
# objective value would be $192,000 lower

firstAllocation_secondROI = sum(obj2[i] * Mod.x[i] for i in range(len(obj)))
diff2 = Mod2.objVal - firstAllocation_secondROI
diff2

192000.0

In [ ]:
# The third constraint does not seem useful, if we can invest more than 3 million and get a higher ROI in a specific
# marketing media, then that is exactly what we should do.



##Part 6 - Reinvestment Strategy and Monthly Optimal Allocations


In [ ]:
# First row of ROI Data's Model

print("\n" + "="*90 + "\n")

# Define headers for the objective analysis table
obj_table_headers = ["Variable", "Allowable Increase", "Allowable Decrease"]

# Print the objective analysis table header
print(f"{obj_table_headers[0]:<30} {obj_table_headers[1]:<25} {obj_table_headers[2]:<25}")

# Loop through the objective coefficients and print the sensitivity analysis results
for platform in platforms:
    allowable_increase = round(float(ModX[platforms[platform]].SAObjUp), 3)
    allowable_decrease = round(float(ModX[platforms[platform]].SAObjLow), 3)
    # Format and print each row of the objective analysis table
    print(f"{platform:<30} {allowable_increase:<25} {allowable_decrease:<25}")



Variable                       Allowable Increase        Allowable Decrease       
Print                          0.049                     -inf                     
TV                             0.062                     0.039                    
SEO                            0.039                     -inf                     
AdWords                        0.046                     0.033                    
Facebook                       0.029                     -inf                     
LinkedIn                       0.039                     -inf                     
Instagram                      inf                       0.039                    
Snapchat                       0.039                     -inf                     
Twitter                        0.039                     -inf                     
Email                          inf                       0.029                    


In [ ]:
# Second row of ROI Data's Model

print("\n" + "="*90 + "\n")

# Define headers for the objective analysis table
obj_table_headers = ["Variable", "Allowable Increase", "Allowable Decrease"]

# Print the objective analysis table header
print(f"{obj_table_headers[0]:<30} {obj_table_headers[1]:<25} {obj_table_headers[2]:<25}")

# Loop through the objective coefficients and print the sensitivity analysis results
for platform in platforms:
    allowable_increase = round(float(ModX2[platforms[platform]].SAObjUp), 3)
    allowable_decrease = round(float(ModX2[platforms[platform]].SAObjLow), 3)
    # Format and print each row of the objective analysis table
    print(f"{platform:<30} {allowable_increase:<25} {allowable_decrease:<25}")



Variable                       Allowable Increase        Allowable Decrease       
Print                          0.052                     0.039                    
TV                             0.049                     -inf                     
SEO                            0.039                     -inf                     
AdWords                        0.046                     0.038                    
Facebook                       inf                       0.029                    
LinkedIn                       inf                       0.039                    
Instagram                      0.039                     -inf                     
Snapchat                       0.039                     -inf                     
Twitter                        0.039                     -inf                     
Email                          0.029                     -inf                     


##Part 7 - Stability Analysis of Monthly Allocation

In [ ]:
roi_mat

,Unnamed: 0,Print,TV,SEO,AdWords,Facebook,LinkedIn,Instagram,Snapchat,Twitter,Email
0,January,4.00,3.60,2.40,3.90,3.00,3.50,3.60,2.25,3.50,3.50
1,February,4.00,3.90,2.70,3.80,4.30,3.20,2.70,1.80,3.70,3.50
2,March,3.50,2.90,3.10,3.80,2.40,4.10,3.70,2.60,4.20,2.50
3,April,3.80,3.10,2.40,4.40,2.40,3.80,3.70,2.50,3.60,2.90
4,May,3.50,3.20,1.90,3.40,2.70,2.70,3.90,2.20,4.50,3.90
5,June,4.00,3.20,2.70,3.40,3.40,3.00,4.50,2.10,3.80,4.10
6,July,3.90,3.60,2.00,4.40,3.90,3.70,4.30,1.80,4.00,3.80
7,August,4.20,3.30,2.80,4.20,2.00,3.70,3.60,1.50,4.40,4.30
8,September,4.10,2.80,2.50,4.20,2.90,3.70,2.80,2.50,4.00,3.40
9,October,3.00,3.00,3.10,4.60,3.10,3.30,3.20,2.30,2.50,3.20


In [ ]:
months = roi_mat.iloc[:, 0]
obj3 = roi_mat.iloc[:, 1:]

In [ ]:
percent_obj3 = obj3 / 100

percent_obj3

,Print,TV,SEO,AdWords,Facebook,LinkedIn,Instagram,Snapchat,Twitter,Email
0,0.04,0.04,0.02,0.04,0.03,0.04,0.04,0.02,0.04,0.04
1,0.04,0.04,0.03,0.04,0.04,0.03,0.03,0.02,0.04,0.04
2,0.04,0.03,0.03,0.04,0.02,0.04,0.04,0.03,0.04,0.03
3,0.04,0.03,0.02,0.04,0.02,0.04,0.04,0.03,0.04,0.03
4,0.04,0.03,0.02,0.03,0.03,0.03,0.04,0.02,0.04,0.04
5,0.04,0.03,0.03,0.03,0.03,0.03,0.04,0.02,0.04,0.04
6,0.04,0.04,0.02,0.04,0.04,0.04,0.04,0.02,0.04,0.04
7,0.04,0.03,0.03,0.04,0.02,0.04,0.04,0.01,0.04,0.04
8,0.04,0.03,0.03,0.04,0.03,0.04,0.03,0.03,0.04,0.03
9,0.03,0.03,0.03,0.05,0.03,0.03,0.03,0.02,0.03,0.03


In [ ]:
# Identifying the indices of each of the platforms in roi_mat
# We are assuming that the platforms included in the constraints are always included in the data
# We also assume that all the column names are spelled & capitalized correctly

platforms2 = {}
columns = roi_mat.iloc[:, 1:]
for column in columns:
    platforms2[column] = columns.columns.get_loc(column)
print(platforms2)

{'Print': 0, 'TV': 1, 'SEO': 2, 'AdWords': 3, 'Facebook': 4, 'LinkedIn': 5, 'Instagram': 6, 'Snapchat': 7, 'Twitter': 8, 'Email': 9}


In [ ]:
Monthly_Budget = 10000000

num_months = len(obj3)
num_mediums = len(obj3.iloc[0])

allocation_results = np.zeros((num_months, num_mediums))

for i in range(0, num_months):
    # Create Model
    Mod3 = gp.Model()

    # Add Decision Variables
    ModX3 = Mod3.addMVar(num_mediums, ub = individual_max)

    # Add Constraints
    Mod3.addConstr(gp.quicksum(ModX3) <= Monthly_Budget)
    Mod3.addConstr((ModX3[platforms2["Print"]] + ModX3[platforms2["TV"]]) <= (ModX3[platforms["Facebook"]] + ModX3[platforms["Email"]]))
    Mod3.addConstr((ModX3[platforms["Facebook"]] + ModX3[platforms["LinkedIn"]] + ModX3[platforms["Instagram"]] + ModX3[platforms["Snapchat"]] + ModX3[platforms["Twitter"]]) >= (2 * (ModX3[platforms["SEO"]] + ModX3[platforms["AdWords"]])))

    # Set Objective
    Mod3.setObjective(gp.quicksum(percent_obj3.iloc[i][j] * ModX3[j] for j in range(num_mediums)), sense = gp.GRB.MAXIMIZE)

    # Stop output and optimize model
    Mod3.Params.OutputFlag = 0
    Mod3.optimize()

    # Update new monthly budgets
    Monthly_Budget += (Mod3.objVal * .5)

    # Append results to a new dataframe
    allocation_results[i] = ModX3.x

In [ ]:
allocation_df = pd.DataFrame(allocation_results)

# Rename the rows to be month names January - December

allocation_df.index = months.tolist()

# Rename the columns to be the marketing mediums

allocation_df.columns = percent_obj3.columns.tolist()

# Format the DataFrame to remove scientific notation

pd.options.display.float_format = '{:,.2f}'.format

# Print the DataFrame

allocation_df

,Print,TV,SEO,AdWords,Facebook,LinkedIn,Instagram,Snapchat,Twitter,Email
January,"3,000,000.00",0.00,0.00,"1,333,333.33",0.00,0.00,"2,666,666.67",0.00,0.00,"3,000,000.00"
February,"3,000,000.00",0.00,0.00,"2,395,500.00","3,000,000.00",0.00,0.00,0.00,"1,791,000.00",0.00
March,0.00,0.00,0.00,"3,000,000.00",0.00,"3,000,000.00","1,389,648.00",0.00,"3,000,000.00",0.00
April,0.00,0.00,0.00,"3,000,000.00",0.00,"3,000,000.00","3,000,000.00",0.00,"1,596,856.49",0.00
May,"1,804,099.90",0.00,0.00,0.00,0.00,0.00,"3,000,000.00",0.00,"3,000,000.00","3,000,000.00"
June,"3,000,000.00",0.00,0.00,0.00,0.00,0.00,"3,000,000.00",0.00,"2,020,171.65","3,000,000.00"
July,0.00,0.00,0.00,"3,000,000.00","2,247,554.91",0.00,"3,000,000.00",0.00,"3,000,000.00",0.00
August,"3,000,000.00",0.00,0.00,"1,827,294.08",0.00,"654,588.16",0.00,0.00,"3,000,000.00","3,000,000.00"
September,"1,362,932.65",0.00,0.00,"3,000,000.00",0.00,"3,000,000.00",0.00,0.00,"3,000,000.00","1,362,932.65"
October,0.00,0.00,0.00,"3,000,000.00",0.00,"3,000,000.00","3,000,000.00",0.00,0.00,"2,955,475.27"


In [ ]:
# A stable budget is defined as a monthly allocation such that for each platform the monthly change in
# spending is no more than $1M. The allocation that we found is not stable.

# The allocation that we found is not stable, as spending drastically changes between months, such as between
# February and March for Print, spending goes from $3,000,000 to $0 (we can also see standard deviations greater than
# 1,000,000). To create a new model with a stable budget, you can add a constraint to the model that specifies that
# the change from month to month for each of our marketing mediums cannot be greater than $1 Million.

In [ ]:
# Show summary statistics for the DataFrame

allocation_df.describe()

,Print,TV,SEO,AdWords,Facebook,LinkedIn,Instagram,Snapchat,Twitter,Email
count,12.00,12.00,12.00,12.00,12.00,12.00,12.00,12.00,12.00,12.00
mean,"1,763,919.38","250,000.00",0.00,"1,920,041.59","687,296.24","1,147,285.84","1,838,026.22",0.00,"1,700,669.01","1,859,867.33"
std,"1,406,976.87","866,025.40",0.00,"1,207,118.82","1,257,088.61","1,410,084.63","1,430,262.15",0.00,"1,352,822.78","1,448,608.79"
min,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,0.00,0.00,0.00,"1,106,987.66",0.00,0.00,0.00,0.00,0.00,0.00
50%,"2,402,049.95",0.00,0.00,"2,225,960.48",0.00,"327,294.08","2,833,333.33",0.00,"1,905,585.83","2,977,737.63"
75%,"3,000,000.00",0.00,0.00,"3,000,000.00","561,888.73","3,000,000.00","3,000,000.00",0.00,"3,000,000.00","3,000,000.00"
max,"3,000,000.00","3,000,000.00",0.00,"3,000,000.00","3,000,000.00","3,000,000.00","3,000,000.00",0.00,"3,000,000.00","3,000,000.00"
